# Less-Referenced Mosaic

This notebook provides a cross-section of the Less-Referenced Mosaic creation process.

# Setup

## Imports

In [ ]:
import copy
import os

In [ ]:
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.utils import check_random_state
import yaml

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
sns.set_style('white')

In [ ]:
from night_horizons import utils, preprocess, reference, mosaic, raster, pipelines

## Settings

In [ ]:
with open('./config.yml', "r", encoding='UTF-8') as file:
    settings = yaml.load(file, Loader=yaml.FullLoader)

In [ ]:
local_settings = {
    'mosaic_filepath': 'mosaics/referenced3.tiff',
    'random_state': 16849,
    'train_size': 1,

    # This set of choices assumes we have really good starting positions.
    # This is useful for debugging.
    # 'padding_factor': 0.05,
    'padding_factor': 0.1,
    'use_approximate_georeferencing': False,
}
settings.update(local_settings)

## Parse Settings

In [ ]:
settings['mosaic_filepath'] = os.path.join(settings['data_dir'], settings['mosaic_filepath'])

In [ ]:
for key, relpath in settings['paths_relative_to_data_dir'].items():
    settings[key] = os.path.join(settings['data_dir'], relpath)

In [ ]:
random_state = check_random_state(settings['random_state'])

In [ ]:
referenced_fps = utils.discover_data(settings['referenced_images_dir'], ['tif', 'tiff'], pattern=r'Geo\s\d+_\d.tif')

In [ ]:
palette = sns.color_palette(settings['color_palette'])

In [ ]:
crs = settings['crs']

# Prepare Data
The first part is to prepare the data (AKA extract/transform/load).

## Train-Test Split

We split the data into training data (data that is georeferenced) and test data (data that is not georeferenced, or for which we don't use the georeferencing information when we're building the models).

We set the train size to some small number, because ideally the user only needs to georeference a couple of images manually.

In [ ]:
settings['train_size']

In [ ]:
fps_train, fps_test = train_test_split(referenced_fps, train_size=settings['train_size'], random_state=settings['random_state'])

## Initial, Approximate Georeferencing
We use the sensor (high-altitude balloon) positions to provide approximate georeferencing, which will be useful for saving computational time when building the unreferenced mosaic.

In [ ]:
# This is the pipeline for approximate georeferencing
sensor_georeference_pipeline = Pipeline([
    ('nitelite', preprocess.NITELitePreprocesser(
        output_columns=['filepath', 'sensor_x', 'sensor_y', 'camera_num'],
        crs=crs
    )),
    ('sensor_georeference', reference.SensorGeoreferencer(crs=crs, passthrough=['filepath', 'camera_num'])),
])

sensor_georeference_pipeline_y = preprocess.GeoTIFFPreprocesser(crs=crs)

In [ ]:
# Get the geo-transforms used for training
y_train = sensor_georeference_pipeline_y.fit_transform(fps_train)
y_test = sensor_georeference_pipeline_y.fit_transform(fps_test)

In [ ]:
# Train the pipeline
sensor_georeference_pipeline.fit(
    fps_train,
    y_train,
    nitelite__img_log_fp=settings['img_log_fp'],
    nitelite__imu_log_fp=settings['imu_log_fp'],
    nitelite__gps_log_fp=settings['gps_log_fp'],
)

In [ ]:
# Get the approximate georeferences for the data we'll be testing
if settings['use_approximate_georeferencing']:
    X_test = sensor_georeference_pipeline.predict(fps_test)
else:
    georeference_pipeline = Pipeline([
        ('nitelite', preprocess.NITELitePreprocesser(
            output_columns=['filepath', 'camera_num'],
            crs=crs,
            unhandled_files='warn and drop',
        )),
        ('georeference', preprocess.GeoTIFFPreprocesser(crs=crs, passthrough=['camera_num'])),
    ])
    X_test = georeference_pipeline.fit_transform(
        fps_test,
        nitelite__img_log_fp=settings['img_log_fp'],
        nitelite__imu_log_fp=settings['imu_log_fp'],
        nitelite__gps_log_fp=settings['gps_log_fp'],
    )

In [ ]:
# Drop the files that were bad from the test data entirely
fps_test = fps_test.loc[X_test.index]
y_test = y_test.loc[X_test.index]

In [ ]:
# The score for the fit gives us an estimate of the error from the approximate georeferencing
first_pass_error = sensor_georeference_pipeline.score(fps_train, y_train)
padding = settings['padding_factor'] * first_pass_error

In [ ]:
# Check that our test Xs and ys align
n_bad = (y_test['filepath'] != X_test['filepath']).sum()
assert n_bad == 0, f'{n_bad} wrong filepaths'

# The Mosaic

### Initialization

#### Test
Check that initialization works, first with a mosaic that only uses the training data.

In [ ]:
small_less_reffed_mosaic = mosaic.LessReferencedMosaic(
    filepath=settings['mosaic_filepath'],
    padding=padding,
    file_exists='overwrite',
)

In [ ]:
small_less_reffed_mosaic.fit(
    X=y_train,
    approx_y=y_train,
)

In [ ]:
# The full mosaic image that's saved
mosaic_img = small_less_reffed_mosaic.dataset_.ReadAsArray().transpose(1, 2, 0)
mosaic_image = raster.ReferencedImage(
    mosaic_img[:, :, :3],
    [small_less_reffed_mosaic.x_min_, small_less_reffed_mosaic.x_max_],
    [small_less_reffed_mosaic.y_min_, small_less_reffed_mosaic.y_max_]
)

In [ ]:
# The actual image used to make it
original_image = raster.ReferencedImage.open(y_train.iloc[0]['filepath'])

In [ ]:
# Compare the mosaic to the actual
mosaic_image.show(crs='cartesian', img='semitransparent_img')

fig = plt.gcf()
ax = plt.gca()

original_image.show(crs='cartesian', img='semitransparent_img', ax=ax)   

In [ ]:
# Check the centers
mosaic_center = np.array(mosaic_image.cart_bounds).mean(axis=1)
original_center = np.array(original_image.cart_bounds).mean(axis=1)
d_between_centers = np.linalg.norm(mosaic_center - original_center)
np.testing.assert_allclose(d_between_centers, 0.)

In [ ]:
# Check the widths
mosaic_width, mosaic_height = np.diff(mosaic_image.cart_bounds, axis=1).flatten()
original_width, original_height = np.diff(original_image.cart_bounds, axis=1).flatten()
np.testing.assert_allclose(mosaic_width, original_width + 2. * padding)
np.testing.assert_allclose(mosaic_height, original_height + 2. * padding)

#### Actual full initialization and fit

In [ ]:
less_reffed_mosaic = mosaic.LessReferencedMosaic(
    filepath=settings['mosaic_filepath'],
    padding=padding,
    file_exists='overwrite',
)

In [ ]:
# This creates the dataset and adds the referenced mosaic.
less_reffed_mosaic.fit(
    X=y_train,
    approx_y=X_test[['filepath'] + preprocess.GEOTRANSFORM_COLS],
)

## Convert geotransforms to pixel offsets and counts

In [ ]:
(
    X_test['x_off'], X_test['y_off'],
    X_test['x_size'], X_test['y_size']
) = less_reffed_mosaic.physical_to_pixel(
    X_test['x_min'], X_test['x_max'],
    X_test['y_min'], X_test['y_max'],
    padding = less_reffed_mosaic.padding
)

In [ ]:
(
    y_train['x_off'], y_train['y_off'],
    y_train['x_size'], y_train['y_size']
) = less_reffed_mosaic.physical_to_pixel(
    y_train['x_min'], y_train['x_max'],
    y_train['y_min'], y_train['y_max'],
    padding = less_reffed_mosaic.padding
)

## Determine order of iteration

In [ ]:
# Camera order of iteration--1 is the nader camera, so that's first
X_test['camera_order'] = X_test['camera_num'].map({0: 1, 1: 0, 2: 2})

In [ ]:
# Proximity order of iteration
center_coords = y_train[['x_center', 'y_center']].mean()
offset = X_test[['x_center', 'y_center']] - center_coords
X_test['d_to_center'] = np.linalg.norm(offset, axis=1)

In [ ]:
# Actual sort
X_iter = X_test.sort_values(['camera_order', 'd_to_center'])
X_iter['order'] = np.arange(len(X_iter))
iter_inds = X_iter.index

In [ ]:
# Let's take a look.
sp = sns.scatterplot(
    data=X_iter,
    x='x_center',
    y='y_center',
    hue='order',
)
sp.set_aspect('equal')

## First Image
We'll test the first loop in greater detail than the others.

In [ ]:
i = 0
row = X_iter.iloc[i]

In [ ]:
mosaic_img = less_reffed_mosaic.dataset_.ReadAsArray().transpose(1, 2, 0)

### Search Region in the Context of the Full Mosaic

In [ ]:
# Expected bounds
x_off = row['x_off']
y_off = row['y_off']
x_size = row['x_size']
y_size = row['y_size']

In [ ]:
(
x_off_nopad, y_off_nopad,
x_size_nopad, y_size_nopad,
) = less_reffed_mosaic.physical_to_pixel(
    row['x_min'], row['x_max'],
    row['y_min'], row['y_max'],
)

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = plt.gca()

# Current mosaic
ax.imshow(mosaic_img)

# The first image location
rect = patches.Rectangle(
    (x_off, y_off),
    x_size,
    y_size,
    linewidth = 3,
    facecolor = 'none',
    edgecolor = palette[0],
)
ax.add_patch(rect)

# The non-padded first image location
rect = patches.Rectangle(
    (x_off_nopad, y_off_nopad),
    x_size_nopad,
    y_size_nopad,
    linewidth = 3,
    facecolor = 'none',
    edgecolor = palette[1],
)
ax.add_patch(rect)

# ax.set_xlim(6000, 10000)
# ax.set_ylim(13000, 9000)

ax.set_aspect('equal')

### Search Region Image 

In [ ]:
# The existing mosaic at this location
dst_img = less_reffed_mosaic.get_image(x_off, y_off, x_size, y_size)

In [ ]:
# At this time we expect all data added to the mosaic to be within the bounds of the search region, if we're using approximate georeferencing
if settings['use_approximate_georeferencing']:
    assert dst_img.sum() == mosaic_img.sum()

This is plotted below with matched features.

In [ ]:
# Here's a zoomed in version so we know what we should be looking at.
row_train = y_train.iloc[0]
zoom_dst_img = less_reffed_mosaic.get_image(
    row_train['x_off'], row_train['y_off'],
    row_train['x_size'], row_train['y_size']
)
plt.imshow(zoom_dst_img)

### Search Region KeyPoints
We get these for later.

In [ ]:
# Get the features from the original mosaic
dst_kp, dst_des = less_reffed_mosaic.feature_detector_.detectAndCompute(dst_img, None)

In [ ]:
# Transform the dst keypoints to mosaic frame
dsframe_dst_pts = cv2.KeyPoint_convert(dst_kp) + np.array([x_off, y_off])
dsframe_dst_des = copy.copy(dst_des)

### New Image

In [ ]:
src_img = utils.load_image(
    row['filepath'],
    dtype=less_reffed_mosaic.dtype,
)

This is plotted below with matched features.

### Feature Matching

In [ ]:
src_kp, src_des = less_reffed_mosaic.feature_detector_.detectAndCompute(src_img, None)

In [ ]:
# Get and validate the transform predicted from feature matching
M, info = utils.calc_warp_transform(src_kp, src_des, dst_kp, dst_des)
assert utils.validate_warp_transform(M, less_reffed_mosaic.homography_det_min)

In [ ]:
# Inspect relationship
mask = info['mask'].reshape(info['mask'].size).astype(bool)
valid_src_pts = info['matched_src_pts'].reshape((mask.size, 2))[mask]
valid_dst_pts = info['matched_dst_pts'].reshape((mask.size, 2))[mask]

In [ ]:
subplot_mosaic = [['dst_img', 'src_img']]
fig = plt.figure(figsize=(20,10))
ax_dict = fig.subplot_mosaic(subplot_mosaic)

ax = ax_dict['dst_img']
ax.imshow(dst_img)

ax = ax_dict['src_img']
ax.imshow(src_img)

for i in range(valid_src_pts.shape[0]):

    con = patches.ConnectionPatch(
        xyA=valid_dst_pts[i],
        xyB=valid_src_pts[i],
        coordsA='data',
        coordsB='data',
        axesA=ax_dict['dst_img'],
        axesB=ax_dict['src_img'],
        color=palette[1],
        linewidth=3,
    )
    ax.add_artist(con)

The feature matching above should look pretty good. It does as I'm writing this.

### Warp the Source Image

In [ ]:
# Warp the image being fit
warped_img = utils.warp_image(src_img, dst_img, M)

In [ ]:
raster.Image(warped_img[:, :, :3]).show(img='semitransparent_img')

In [ ]:
# The warped image should have the same dimensions as the dst img
assert warped_img.shape[:2] == dst_img.shape[:2]

### Blend the images

In [ ]:
blended_img = less_reffed_mosaic.blend_images(
    src_img=warped_img,
    dst_img=dst_img,
)

### Save and look at the mosaic

In [ ]:
less_reffed_mosaic.save_image(blended_img, x_off, y_off)

In [ ]:
# Get the region of just the first image for comparison from before
zoom_dst_img_after = less_reffed_mosaic.get_image(
    row_train['x_off'], row_train['y_off'],
    row_train['x_size'], row_train['y_size'],
)

In [ ]:
# More content should have been added
assert zoom_dst_img_after.sum() > zoom_dst_img.sum()

In [ ]:
# View
subplot_mosaic = [['before', 'after']]
fig = plt.figure(figsize=(20,10))
ax_dict = fig.subplot_mosaic(subplot_mosaic)

ax = ax_dict['before']
raster.Image(zoom_dst_img[:, :, :3]).show(img='semitransparent_img', ax=ax)

ax = ax_dict['after']
raster.Image(zoom_dst_img_after[:, :, :3]).show(img='semitransparent_img', ax=ax)

### Warp the Keypoints

In [ ]:
# Transform to local frame and then the full mosaic frame
src_pts = cv2.KeyPoint_convert(src_kp)
global_src_pts = cv2.perspectiveTransform(src_pts.reshape(-1, 1, 2), M).reshape(-1, 2)
global_src_pts += np.array([x_off, y_off])

In [ ]:
# Store the transformed points for the next loop
dsframe_dst_pts = np.append(dsframe_dst_pts, global_src_pts, axis=0)
dsframe_dst_des = np.append(dsframe_dst_des, src_des, axis=0)

In [ ]:
fig = plt.figure()
ax = plt.gca()

sns.scatterplot(
    x=dsframe_dst_pts[:,0],
    y=dsframe_dst_pts[:,1],
    ax = ax,
)

rect = patches.Rectangle(
    (x_off, y_off),
    x_size,
    y_size,
    linewidth = 3,
    facecolor = 'none',
    edgecolor = palette[0],
)
ax.add_patch(rect)

ax.set_xlim(0, less_reffed_mosaic.dataset_.RasterXSize)
ax.set_ylim(less_reffed_mosaic.dataset_.RasterYSize, 0)
ax.set_aspect('equal')

In [ ]:
# Automated check that everything's in bounds
not_in_bounds = ~(
    (x_off <= dsframe_dst_pts[:,0] )
    & (dsframe_dst_pts[:,0] <= x_off + x_size)
    & (y_off <= dsframe_dst_pts[:,1] )
    & (dsframe_dst_pts[:,1] <= y_off + y_size)
)
assert not_in_bounds.sum() == 0

### Check the georeferencing

In [ ]:
# Call the fn
warped_x_off, warped_y_off, warped_x_size, warped_y_size = utils.warp_bounds(src_img, M)
warped_x_off += x_off
warped_y_off += y_off

In [ ]:
# Convert to physical
warped_x_min, warped_x_max, warped_y_min, warped_y_max = less_reffed_mosaic.pixel_to_physical(
    warped_x_off, warped_y_off, warped_x_size, warped_y_size)

In [ ]:
# Get the recorded bounds
recorded_x_min, recorded_x_max, recorded_y_min, recorded_y_max = y_test.loc[row.name, ['x_min', 'x_max', 'y_min', 'y_max']]

In [ ]:
# Get the centers
warped_center = np.array([
    0.5 * (warped_x_min + warped_x_max),
    0.5 * (warped_y_min + warped_y_max),
])
recorded_center = np.array([
    0.5 * (recorded_x_min + recorded_x_max),
    0.5 * (recorded_y_min + recorded_y_max),
])

In [ ]:
# Check the centers
assert np.linalg.norm(warped_center - recorded_center) < 100.

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = plt.gca()

# The warped image location
width = warped_x_max - warped_x_min
height = warped_y_max - warped_y_min
rect = patches.Rectangle(
    (warped_x_min, warped_y_min),
    width,
    height,
    linewidth = 3,
    facecolor = 'none',
    edgecolor = palette[0],
)
ax.add_patch(rect)
ax.scatter(
    *warped_center,
    s=100,
    color=palette[0],
)

# The actual image location
rect = patches.Rectangle(
    (recorded_x_min, recorded_y_min),
    recorded_x_max - recorded_x_min,
    recorded_y_max - recorded_y_min,
    linewidth = 3,
    facecolor = 'none',
    edgecolor = palette[1],
)
ax.add_patch(rect)
ax.scatter(
    *recorded_center,
    s=100,
    color=palette[1],
)

padding_for_this_plot = 0.1 * width
ax.set_xlim(warped_x_min - padding_for_this_plot, warped_x_max + padding_for_this_plot)
ax.set_ylim(warped_y_min - padding_for_this_plot, warped_y_max + padding_for_this_plot)

ax.set_aspect('equal')

## Next Image

In [ ]:
i = 1
row = X_iter.iloc[i]

### Preview keypoint selection

In [ ]:
x_off = row['x_off']
y_off = row['y_off']
x_size = row['x_size']
y_size = row['y_size']

In [ ]:
in_bounds = less_reffed_mosaic.check_bounds(
    dsframe_dst_pts,
    x_off, y_off, x_size, y_size,
)

In [ ]:
assert in_bounds.sum() > 0, \
    f'No image data in the search zone for index {row.name}'

In [ ]:
dst_pts = dsframe_dst_pts[in_bounds]
dst_des = dsframe_dst_des[in_bounds]

In [ ]:
# At this point in the loops, *all* the points should be in bounds, if we're doing approximate georeferencing
if settings['use_approximate_georeferencing']:
    assert (~in_bounds).sum() == 0

### Call the typical function

In [ ]:
return_code, info = less_reffed_mosaic.incorporate_image(
    row,
    dsframe_dst_pts,
    dsframe_dst_des,
)

In [ ]:
assert return_code == 0, 'Image combined unsuccessfully.'

In [ ]:
zoom_dst_img_after2 = less_reffed_mosaic.get_image(
    row_train['x_off'], row_train['y_off'],
    row_train['x_size'], row_train['y_size'],
)

In [ ]:
subplot_mosaic = [['before', 'after']]
fig = plt.figure(figsize=(20,10))
ax_dict = fig.subplot_mosaic(subplot_mosaic)

ax = ax_dict['before']
raster.Image(zoom_dst_img_after[:, :, :3]).show(img='semitransparent_img', ax=ax)

ax = ax_dict['after']
raster.Image(zoom_dst_img_after2[:, :, :3]).show(img='semitransparent_img', ax=ax)

### Check georeferencing

In [ ]:
# Get the recorded bounds
recorded_x_min, recorded_x_max, recorded_y_min, recorded_y_max = y_test.loc[row.name, ['x_min', 'x_max', 'y_min', 'y_max']]

In [ ]:
# Get the warped bounds
(
    warped_x_min, warped_x_max,
    warped_y_min, warped_y_max,
) = less_reffed_mosaic.pixel_to_physical(
    info['x_off'], info['y_off'],
    info['x_size'], info['y_size'],
)

In [ ]:
# Get the centers
warped_center = np.array([
    0.5 * (warped_x_min + warped_x_max),
    0.5 * (warped_y_min + warped_y_max),
])
recorded_center = np.array([
    0.5 * (recorded_x_min + recorded_x_max),
    0.5 * (recorded_y_min + recorded_y_max),
])

In [ ]:
# Check the centers
d_between_centers = np.linalg.norm(warped_center - recorded_center)
assert d_between_centers < 100.

In [ ]:
src_image = raster.ReferencedImage.open(row['filepath'])

In [ ]:
blended_img = less_reffed_mosaic.get_image(row['x_off'], row['y_off'], row['x_size'], row['y_size'])
blended_image = raster.ReferencedImage(blended_img[:, :, :3], [row['x_min'], row['x_max']], [row['y_min'], row['y_max']])

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = plt.gca()

blended_image.show(crs='cartesian', img='semitransparent_img', ax=ax)

src_image.show(crs='cartesian', img='semitransparent_img', ax=ax)

# The warped image location
width = warped_x_max - warped_x_min
height = warped_y_max - warped_y_min
rect = patches.Rectangle(
    (warped_x_min, warped_y_min),
    width,
    height,
    linewidth = 3,
    facecolor = 'none',
    edgecolor = palette[0],
)
ax.add_patch(rect)
ax.scatter(
    *warped_center,
    s=100,
    color=palette[0],
    label='found',
)

# The actual image location
rect = patches.Rectangle(
    (recorded_x_min, recorded_y_min),
    recorded_x_max - recorded_x_min,
    recorded_y_max - recorded_y_min,
    linewidth = 3,
    facecolor = 'none',
    edgecolor = palette[1],
)
ax.add_patch(rect)
ax.scatter(
    *recorded_center,
    s=100,
    color=palette[1],
    label='recorded',
)

padding_for_this_plot = 0.1 * width
ax.set_xlim(warped_x_min - padding_for_this_plot, warped_x_max + padding_for_this_plot)
ax.set_ylim(warped_y_min - padding_for_this_plot, warped_y_max + padding_for_this_plot)

ax.set_aspect('equal')
ax.legend()

## Run for a Subset

Now we'll check if it runs for a subset

In [ ]:
i = 0
n_loops = 10
iter_inds_subset = iter_inds[i:i + n_loops]
X_iter = X_test.loc[iter_inds_subset]

In [ ]:
less_reffed_mosaic = mosaic.LessReferencedMosaic(
    filepath=settings['mosaic_filepath'],
    padding=padding,
    file_exists='overwrite',
    feature_detector_kwargs={},
)

less_reffed_mosaic.fit(
    X=y_train[['filepath'] + preprocess.GEOTRANSFORM_COLS],
    approx_y=X_iter[['filepath'] + preprocess.GEOTRANSFORM_COLS],
)

y_pred = less_reffed_mosaic.predict(
    X_iter[['filepath', ] + preprocess.GEOTRANSFORM_COLS],
    iteration_indices=iter_inds_subset
)

less_reffed_mosaic.close()

In [ ]:
n_bad = y_pred.isna().sum().sum()
assert n_bad == 0, f'Found {n_bad} nan values, i.e. {n_bad//len(y_pred.columns)} rows.'

In [ ]:
# Estimate the consistency with the manual geotransforms
y_test = sensor_georeference_pipeline_y.fit_transform(fps_test)
y_iter = y_test.loc[iter_inds_subset]
y_err = y_iter - y_pred
err = np.sqrt(y_err['x_min']**2. + y_err['y_max']**2.)

In [ ]:
# Check how bad the errors are
n_egregious = (err > 200.).sum()
assert n_egregious == 0, f'Found {n_egregious} egregious errors.'

In [ ]:
# Visualize the errors
fig = plt.figure()
ax = plt.gca()

sns.scatterplot(
    x=np.arange(y_err.index.size),
    y=err,
    hue=np.arange(len(y_err)),
    ax=ax,
)

ax.set_ylim(0, ax.get_ylim()[1])

# Hyperparameter exploration

In [ ]:
import itertools
import tqdm.notebook

# Input: dictionary of lists
input_dict = {
    'nfeatures': [500,],
    'patchSize': [31, 51, 101],
    'nlevels': [2, 4, 8],
    'firstLevel': [0, 2, 4, 6, 8],
    'WTA_K': [3, 4],
}

# Generate all permutations of values
keys, values = zip(*input_dict.items())
permutations = itertools.product(*values)

# Convert permutations into a list of dictionaries
list_of_kwargs = [dict(zip(keys, permutation)) for permutation in permutations]

In [ ]:
# Loop to see if there's some combination of parameters we can tweak that works better
n_bad = []
last_i = []
for i, kwargs in enumerate(tqdm.notebook.tqdm(list_of_kwargs)):

    print(f'i={i}')
 
    less_reffed_mosaic = mosaic.LessReferencedMosaic(
        filepath=settings['mosaic_filepath'],
        padding=padding,
        file_exists='overwrite',
        feature_detector_kwargs=kwargs,
        verbose=False,
    )
    
    less_reffed_mosaic.fit(
        X=y_train[['filepath'] + preprocess.GEOTRANSFORM_COLS],
        approx_y=X_test[['filepath'] + preprocess.GEOTRANSFORM_COLS],
    )

    # Actual call
    try:
        less_reffed_mosaic.predict(
            X_test[['filepath', ] + preprocess.GEOTRANSFORM_COLS],
            iteration_indices=iter_inds_subset
        )
    
        n_bad.append(len(less_reffed_mosaic.log_['bad_inds']))
    except:
        n_bad.append(np.nan)

    last_i.append(less_reffed_mosaic.log_['last_i'])

In [ ]:
n_bad = np.array(n_bad)
good_inds = np.arange(n_bad.size)[n_bad == 0]

In [ ]:
pd.Series(n_bad).unique()

In [ ]:
(n_bad==0).sum()

# DEBUG

#### Specific Bad Ind

In [ ]:
i = n_loops
row = X_iter.iloc[i]

#### Search Region in the Context of the Full Mosaic

In [ ]:
# Expected bounds
x_off = row['x_off']
y_off = row['y_off']
x_size = row['x_size']
y_size = row['y_size']

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = plt.gca()

# Current mosaic
ax.imshow(mosaic_img)

# The first image location
rect = patches.Rectangle(
    (x_off, y_off),
    x_size,
    y_size,
    linewidth = 3,
    facecolor = 'none',
    edgecolor = palette[0],
)
ax.add_patch(rect)

#### Search Region Image and Keypoints

In [ ]:
# The existing mosaic at this location
dst_img = less_reffed_mosaic.get_image(x_off, y_off, x_size, y_size)

In [ ]:
in_bounds = less_reffed_mosaic.check_bounds(
    dsframe_dst_pts,
    x_off, y_off, x_size, y_size,
)

In [ ]:
assert in_bounds.sum() > 0, \
    f'No image data in the search zone for index {row.name}'

In [ ]:
dst_pts = dsframe_dst_pts[in_bounds] - np.array([x_off, y_off])
dst_kp = cv2.KeyPoint_convert(dst_pts)
dst_des = dsframe_dst_des[in_bounds]

#### Image to Add and Keypoints

In [ ]:
src_img = utils.load_image(
    row['filepath'],
    dtype=less_reffed_mosaic.dtype,
)

In [ ]:
src_kp, src_des = less_reffed_mosaic.feature_detector.detectAndCompute(src_img, None)
src_pts = cv2.KeyPoint_convert(src_kp)

#### Side-by-side

In [ ]:
subplot_mosaic = [['dst_img', 'src_img']]
fig = plt.figure(figsize=(20,10))
ax_dict = fig.subplot_mosaic(subplot_mosaic)

ax = ax_dict['dst_img']
ax.imshow(dst_img)
ax.scatter(
    dst_pts[:,0],
    dst_pts[:,1],
    color = palette[0],
)

ax = ax_dict['src_img']
ax.imshow(src_img)
ax.scatter(
    src_pts[:,0],
    src_pts[:,1],
    color = palette[1],
)


#### What if we only included the keypoints that actually might match?

In [ ]:
# in_bounds_src = src_pts[:,1] > 1250
# src_kp = cv2.KeyPoint_convert(src_pts[in_bounds_src])
# src_des = src_des[in_bounds_src]

#### Feature Matching

In [ ]:
# Get and validate the transform predicted from feature matching
M, info = utils.calc_warp_transform(src_kp, src_des, dst_kp, dst_des)
assert utils.validate_warp_transform(M, 1e-4)

#### Incorporation

In [ ]:
# Convert to the dataset frame
src_pts = cv2.KeyPoint_convert(src_kp)
dsframe_src_pts = cv2.perspectiveTransform(
    src_pts.reshape(-1, 1, 2),
    M,
).reshape(-1, 2)
dsframe_src_pts += np.array([x_off, y_off])

# Warp the source image
warped_img = cv2.warpPerspective(src_img, M, (x_size, y_size))

# Combine the images
blended_img = less_reffed_mosaic.blend_images(
    src_img=warped_img,
    dst_img=dst_img,
)

In [ ]:
plt.imshow(blended_img)

In [ ]:
np.linalg.det(M)

### Search Region Keypoints

In [ ]:
# Inspect relationship
mask = info['mask'].reshape(info['mask'].size).astype(bool)
valid_src_pts = info['matched_src_pts'].reshape((mask.size, 2))[mask]
valid_dst_pts = info['matched_dst_pts'].reshape((mask.size, 2))[mask]

In [ ]:
subplot_mosaic = [['dst_img', 'src_img']]
fig = plt.figure(figsize=(20,10))
ax_dict = fig.subplot_mosaic(subplot_mosaic)

ax = ax_dict['dst_img']
ax.imshow(dst_img)

ax = ax_dict['src_img']
ax.imshow(src_img)

for i in range(valid_src_pts.shape[0]):

    con = patches.ConnectionPatch(
        xyA=valid_dst_pts[i],
        xyB=valid_src_pts[i],
        coordsA='data',
        coordsB='data',
        axesA=ax_dict['dst_img'],
        axesB=ax_dict['src_img'],
        color=palette[1],
        linewidth=3,
    )
    ax.add_artist(con)

### DEBUG

In [ ]:
less_reffed_mosaic.close()

In [ ]:
x_off = row['x_off']
y_off = row['y_off']
x_size = row['x_size']
y_size = row['y_size']

In [ ]:
# The empty image at the time of the first loop
mosaic_img = less_reffed_mosaic.dataset_.ReadAsArray().transpose(1, 2, 0)

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = plt.gca()

# Current mosaic
ax.imshow(mosaic_img)

# The first image location
rect = patches.Rectangle(
    (x_off, y_off),
    x_size,
    y_size,
    linewidth = 3,
    facecolor = 'none',
    edgecolor = palette[0],
)
ax.add_patch(rect)

In [ ]:
src_img = utils.load_image(
    row['filepath'],
    dtype=less_reffed_mosaic.dtype,
)
src_kp, src_des = less_reffed_mosaic.feature_detector.detectAndCompute(src_img, None)

In [ ]:
# Get dst features
in_bounds = less_reffed_mosaic.check_bounds(
    dsframe_dst_pts,
    x_off, y_off, x_size, y_size
)
assert in_bounds.sum() > 0, \
    f'No image data in the search zone for index {row.name}'
dst_pts = dsframe_dst_pts[in_bounds] - np.array([x_off, y_off])
dst_kp = cv2.KeyPoint_convert(dst_pts)
dst_des = dsframe_dst_des[in_bounds]

In [ ]:
# Feature matching
M, info = utils.calc_warp_transform(
    src_kp,
    src_des,
    dst_kp,
    dst_des,
    less_reffed_mosaic.feature_matcher,
)
np.linalg.det(M)

In [ ]:
# Inspect relationship
mask = info['mask'].reshape(info['mask'].size).astype(bool)
valid_src_pts = info['matched_src_pts'].reshape((mask.size, 2))[mask]
valid_dst_pts = info['matched_dst_pts'].reshape((mask.size, 2))[mask]

In [ ]:
dst_img = less_reffed_mosaic.get_image(x_min, x_max, y_min, y_max)

In [ ]:
subplot_mosaic = [['dst_img', 'src_img']]
fig = plt.figure(figsize=(20,10))
ax_dict = fig.subplot_mosaic(subplot_mosaic)

ax = ax_dict['dst_img']
ax.imshow(dst_img)

ax = ax_dict['src_img']
ax.imshow(src_img)

for i in range(valid_src_pts.shape[0]):

    con = patches.ConnectionPatch(
        xyA=valid_dst_pts[i],
        xyB=valid_src_pts[i],
        coordsA='data',
        coordsB='data',
        axesA=ax_dict['dst_img'],
        axesB=ax_dict['src_img'],
        color=palette[1],
        linewidth=3,
    )
    ax.add_artist(con)

### DEBUG

In [ ]:
dst_img_zoom_after3 = less_reffed_mosaic.get_image(
    row_train['x_min'], row_train['x_max'],
    row_train['y_min'], row_train['y_max']
)

In [ ]:
subplot_mosaic = [['before', 'after']]
fig = plt.figure(figsize=(20,10))
ax_dict = fig.subplot_mosaic(subplot_mosaic)

ax = ax_dict['before']
ax.imshow(dst_img_zoom_after2)

ax = ax_dict['after']
ax.imshow(dst_img_zoom_after3)

## Full Mosaic

Now that we've checked the process, we'll do the full loop.

In [ ]:
less_reffed_mosaic.predict(X_test[['filepath'] + preprocess.GEOTRANSFORM_COLS], iteration_indices=iter_inds)

In [ ]:
less_reffed_mosaic.log['return_codes']